In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'NanumGothic'
import warnings
warnings.filterwarnings('ignore')

import shap
import scipy as sp

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoModel
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

import tqdm

# GPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device:", device)

device: cuda


In [ ]:
eval_df = pd.read_excel("/home/xogns5037/KcELECTRA/datasets/Final_Evaluation.xlsx")
eval_df

,Raw Sentence,Our Model,w/o fewshot,w/o MASK,Nothing ( just Instruction )
0,씨바알..노무노무 술프노... 오늘 저녁은 꽂등심이다ㅠㅜ,아.. 정말정말 술이 땡기네... 오늘 저녁은 꽃등심이다ㅠㅜ,아 정말 너무 슬퍼... 오늘 저녁은 꽃등심이네요.😢,정말 슬프네요... 오늘 저녁은 꽃등심이에요.,{'정말 슬프네요... 오늘 저녁은 꽃등심이에요.ㅠㅜ\n'}
1,아무리 법이 뭣같아도 무슨 자격으로 개인의 신상정보를 불특정 다수에게 공개하는지 도...,아무리 법이 불공정해도 어떤 권리로 개인의 신상정보를 불특정 다수에게 공개하는지 도...,"법이 완벽하지 않다고 해도, 어떤 권한으로 개인 정보를 여러 사람에게 공개하는지 이...","법이 완벽하지 않을 수 있지만, 어떤 권한으로 개인의 신상 정보를 많은 사람에게 공...","법이 만족스럽지 못할 수 있지만, 개인의 신상 정보를 공개하는 것은 적절한 절차를 ..."
2,다음도 들어와라하고...다른 의견내는 유튜버 목 자르고....추아줌마 꺼 여기저기서...,다음도 들어와라 하고... 다른 의견 내는 유튜버를 배제하고... 추측이 여기저기서...,"다음에도 참여를 요청하며, 다른 의견을 제시하는 유튜버에게 압력을 가하고... 여러...",다음도 참여하라고 하고... 다른 의견을 내는 유튜버에게 압력을 가하고... 추 장...,"다음에도 참여를 요청하고, 다른 의견을 제시하는 유튜버에 대한 조치를 취하며, 여러..."
3,개별 공무원 뷰웅들의 헷지랄에 뭔 청와대까지 들먹이는지... 빠끄네 때도 순시리가 ...,개별 공무원 분들의 활동에 왜 청와대까지 언급하는지 모르겠네요. 이전에도 문제가 있...,공무원들의 행동에 왜 청와대까지 거론되는지 모르겠네요. 이전 정부 때도 문제가 있었...,개별 공무원들의 행동에 왜 청와대까지 언급하는지 모르겠네요... 이전에도 문제는 개...,공무원 개인의 행동을 가지고 청와대까지 언급하는 것은 다소 과한 것 같습니다. 이전...
4,대깨문이 문재인 협박범을 쉴드치네? 역시 대가리가 붕어인듯~,지지자들이 문재인 협박범을 옹호하네? 역시 생각이 단순한 듯~,일부 사람들이 문재인 전 대통령을 지지하는 것 같네요. 역시 의견이 갈리는 모양이네요.,문재인 전 대통령을 지지하는 분들이 협박범을 옹호하고 있네요? 역시 생각이 다른 것...,{'문재인 전 대통령을 지지하는 분들이 협박범을 옹호하는 것 같네요. 의견이 다를 ...
...,...,...,...,...,...
1354,펨코 이 씨발련들 싸가지 왤케 없냐저 정도 길이면 그냥 건너갈 수도 있지누군가의 엄...,펨코 이 사람들 예의가 왜 이렇게 없나요? 저 정도 길이면 그냥 건너갈 수도 있지....,"펨코 여러분, 댓글을 좀 더 예의 있게 달아주셨으면 해요. 저 정도 길이면 건너갈 ...",펨코 이 사람들 예의가 왜 이렇게 없나요? 저 정도 길이면 그냥 건너갈 수도 있잖아...,"펨코 이용자분들, 좀 더 예의를 갖추면 좋겠어요. 저 정도 길이면 건너갈 수도 있을..."
1355,홍어새끼들 존나 많네 이정도 디테일보고 주작주작 거리고 있네 씨발년들아 태어나기전...,"사람들 정말 많네요. 이정도 디테일 보고도 의심하고 있네요. 여러분, 태어나기 전 ...",많은 사람들이 이 정도의 디테일을 보고 의심하고 있는 것 같네요. 태어나기 전 일을...,여기 사람들 정말 많네요. 이 정도 디테일을 보고도 의심이 많으시네요. 태어나기 전...,여기 사람들 정말 많네요. 이렇게 세세한 부분까지 살펴보고 의심하는 분위기가 있네요...
1356,손주은 센세가 떠오르노 ㅋㅋㅋㅋㅋㅋㅋ강의시간에 너 공부 안하면 창녀보다 못한 삶을 ...,손주은 선생님이 생각나네요. 강의시간에 너 공부 안 하면 힘든 삶을 살게 될 거라는...,손주은 선생님이 생각나네요. 강의시간에 열심히 공부하지 않으면 힘든 삶을 살게 될 ...,손주은 선생님이 생각나네요. 강의시간에 공부를 열심히 하지 않으면 어려운 삶을 살게...,손주은 선생님의 강의가 생각나네요. 강의 중에 공부를 열심히 하지 않으면 어려운 삶...
1357,이래서 애새끼 함부로 싸지르는거 아니다,이래서 아이를 함부로 낳는 건 아니다,아이를 함부로 낳는 것은 책임감 있는 행동이 아닙니다.,이래서 아이를 함부로 낳는 것이 아닙니다.,아이를 키우는 일은 신중하게 접근해야 하는 중요한 일입니다.


In [6]:
# MODEL_NAME = "beomi/KcELECTRA-base"
MODEL_NAME = "beomi/KcELECTRA-base-v2022"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [7]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
model.to(device)

Some weights of the model checkpoint at beomi/KcELECTRA-base-v2022 were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.dense.bias', 'c

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(54343, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [ ]:
# 모델 불러오기
model.load_state_dict(torch.load('/home/xogns5037/KcELECTRA/KcELECTRA_5_pth/checkpoint-4500/pytorch_model.bin'))

<All keys matched successfully>

### 전체 평가 지표(finetuning)

In [10]:
# 0: curse, 1: non_curse
def sentence_predict(sent):
    # 평가모드로 변경
    model.eval()

    # 입력된 문장 토크나이징
    tokenized_sent = tokenizer(
        sent,
        return_tensors="pt",
        truncation=True,
        add_special_tokens=True,
        max_length=128
    )
    
    # 모델이 위치한 GPU로 이동 
    tokenized_sent.to(device)

    # 예측
    with torch.no_grad():
        outputs = model(
            input_ids=tokenized_sent["input_ids"],
            attention_mask=tokenized_sent["attention_mask"],
            token_type_ids=tokenized_sent["token_type_ids"]
            )

    # 결과 return
    logits = outputs[0]
    logits = logits.detach().cpu()
    result = logits.argmax(-1)
    if result == 0: # 악성댓글
        result = 0
    elif result == 1: # 정상댓글
        result = 1
    return result


accuracy = {}

for column in eval_df.columns:
    
    pred = 0
    total = len(eval_df[column])

    for sent in eval_df[column]:
        result = sentence_predict(sent)
        # print(f"sentence: {sent}, result: {result}")
        if result == 0:
            continue
        else:
            pred += 1
    accuracy[column] = pred/total

accuracy

{'Raw Sentence': 0.0,
 'Our Model': 0.9323031640912436,
 'w/o fewshot': 0.9926416482707874,
 'w/o MASK': 0.9610007358351729,
 'Nothing ( just Instruction )': 0.9867549668874173}